# Binance Profit Calculator 
`Written by @cryptoDey94 (telegram)` <br/>
Donations Appreciated :)<br/>
BTC Address: 1MLadoaCujyjXZUfbKR4greXiR84SaH4T8 <br/>

## Designed for use with [Gunbot](https://github.com/GuntharDeNiro/BTCT/wiki), but will work for any binance trading history. 


# Usage Instructions


## Download trading history from Binance

Log on to Binance and navigate to `Orders` -> `Trade History`. On the top right should be an option `Export complete trade history`. Click on that, chose a time period, and download the excel file. Make sure this .ipynb fle is in the same folder as `TradeHistory.xlsx`. (If filename is different than that, change the filename in the below cell).

In [1]:
filename = 'TradeHistory.xlsx'

## Run this file

#### Traditional Way
To run this file, I reccomend using Anaconda Navigator. To download, click the link below. Then open Anaconda Navigator, and click `Launch` under `Jupyter notebook`. Your browser will open with a screen to select which .ipynb file to open. Download and open this one, but before running, finish reading instructions. 
https://www.anaconda.com/download/

#### Easy way (Recommended)
Create an empty github repository for free, and just upload your `TradingHistory.xlsx` file and this .ipynb file to your github repo. Since github will only run this once the first time you upload it, to update the results with new trade history file, simply delete both files from github, change this .ipynb file name to a new name and re upload it to github along with the new trading history. Then github will detect it as a new file and re calculate. :)

## Notes on the program

The program calculates profits based on round-trip trades in the given time period. That is, for each coin pair, any sells that come before any buys and any buys that come after any sells will not be calculated. Thus, only full trades (trades that contain a buy and a sell) will be calcuated for profit. This is the most accurate way to determine how much profit you accumulated in the given time period of the exported excel file.

The program calculates fees based on BNB. If you do not use BNB for fees, A. You should since its half the cost and B. you need to set the variable `use_fees` below to `False`. Note, this will calculate total profit without fees included.

In [2]:
use_fees = True

In [3]:
import pandas as pd
import numpy as np
import copy
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

# initialize variables we will need
i = 0
totalProfit = 0.0
rawGroupDfs = {}
brokenDf = pd.DataFrame(columns=['Name', 'Total Bought', 'Total Sold', 'Profit', 'Fee'])
resultDf = pd.DataFrame()

In [4]:
df = pd.read_excel(filename)

df['Date'] = df['Date(UTC)']
end = pd.to_datetime(df['Date'].iloc[0]).strftime('%I:%M %p on %B %d, %Y')
start = pd.to_datetime(df['Date'].iloc[-1]).strftime('%I:%M %p on %B %d, %Y')
print('\n')
print('Will calculate profit from ' + start + ' to ' + end)
print('\n')
print('Successfully Imported Excel File. Printing first 10 rows...')
df.head(10)



Will calculate profit from 11:17 AM on April 28, 2018 to 11:10 AM on April 29, 2018


Successfully Imported Excel File. Printing first 10 rows...


,Date(UTC),Market,Type,Price,Amount,Total,Fee,Fee Coin,Date
0,2018-04-29 11:10:22,IOTABTC,BUY,0.000210,276.00,0.057905,0.018300,BNB,2018-04-29 11:10:22
1,2018-04-29 11:09:01,IOTABTC,SELL,0.000209,269.00,0.056283,0.017770,BNB,2018-04-29 11:09:01
2,2018-04-29 11:09:01,IOTABTC,SELL,0.000209,7.00,0.001465,0.000461,BNB,2018-04-29 11:09:01
3,2018-04-29 11:07:51,IOTABTC,BUY,0.000211,138.00,0.029081,0.009173,BNB,2018-04-29 11:07:51
4,2018-04-29 11:03:14,BTSBTC,BUY,0.000031,471.00,0.014380,0.004530,BNB,2018-04-29 11:03:14
5,2018-04-29 11:03:14,BTSBTC,BUY,0.000031,454.00,0.013861,0.004367,BNB,2018-04-29 11:03:14
6,2018-04-29 11:03:14,BTSBTC,BUY,0.000030,58.00,0.001766,0.000558,BNB,2018-04-29 11:03:14
7,2018-04-29 11:00:22,STEEMBTC,BUY,0.000432,0.01,0.000004,0.000001,BNB,2018-04-29 11:00:22
8,2018-04-29 11:00:16,STEEMBTC,BUY,0.000432,46.82,0.020226,0.006399,BNB,2018-04-29 11:00:16
9,2018-04-29 11:00:15,STEEMBTC,BUY,0.000432,86.91,0.037545,0.011878,BNB,2018-04-29 11:00:15


The below code sanitizes the data and calculates profits based on the complete trades rules we have set above in `O(n)` time. 

In [5]:
xls = df.groupby(df['Market'], as_index=False)
clean_groups = []
#loop start - long cell bc all work being calculated in one go AKA O(n)
for name, group in xls:
    
    group = group.sort_values(by=['Date'], ascending=True)
    rawGroupDfs[name] = copy.deepcopy(group)
    
    # Create group dataframes for profit calculations
    group_df = pd.DataFrame(columns=['Total Bought', 'Total Sold', 'Profit', 'Fee'])

    # Remove sells from bottom
    group = group.reset_index()
    for index, row in group.iterrows():
        if row['Type'] == 'SELL':
            group.drop(index, inplace=True)
        else:
            break

    # Remove buys from top
    while group['Type'].count() > 0 and group.iloc[-1]['Type'] == 'BUY':
        group = group[:-1]


    # check for groups that only sold or bought, dont count them
    if len(group['Type'].unique()) == 1:
        continue
        
    
    # Calculate profits
    totalCostForCurrentGroup = 0.0
    totalRevenueForCurrentGroup = 0.0
    totalProfitForCurrentGroup = 0.0
    totalFeeForCurrentGroup = 0.0
    totalAmountForBuys = 0.0
    totalAmountForSells = 0.0

    for index, row in group.iterrows():
        totalFeeForCurrentGroup = totalFeeForCurrentGroup + float(row['Fee'])
        if row['Type'] == 'BUY':
            totalAmountForBuys = totalAmountForBuys + float(row['Amount'])
            totalCostForCurrentGroup = totalCostForCurrentGroup + float(row['Total'])
            
        if row['Type'] == 'SELL':
            totalAmountForSells = totalAmountForSells + float(row['Amount'])
            totalRevenueForCurrentGroup = totalRevenueForCurrentGroup + float(row['Total'])
        
        


    # sometimes there is a slight difference in the amount bought / sold 
    # (maybe limit orders didnt get fully filled in time or owned coins from prev trades)
    # We will recalculate profits of those groups to be conservative to the lower amount
    group_df = pd.DataFrame()

    if totalAmountForSells > totalAmountForBuys:
        group_df = group_df.set_value(i, 'Only counting', totalAmountForSells if totalAmountForBuys > totalAmountForSells else totalAmountForBuys)
        totalRevenueForCurrentGroup = totalRevenueForCurrentGroup * (totalAmountForBuys / totalAmountForSells)
        totalFeeForCurrentGroup = totalFeeForCurrentGroup * (totalAmountForBuys / totalAmountForSells)
        print('Fixing relative revenue for group ' + name)
        print('\n')
        
        
    if totalAmountForSells < totalAmountForBuys:
        group_df = group_df.set_value(i, 'Only counting', totalAmountForSells if totalAmountForBuys > totalAmountForSells else totalAmountForBuys)        
        totalCostForCurrentGroup = totalCostForCurrentGroup * (totalAmountForSells / totalAmountForBuys)
        totalFeeForCurrentGroup = totalFeeForCurrentGroup * (totalAmountForSells / totalAmountForBuys)
        print('Fixing relative cost for group ' + name)
        print('\n')


    totalProfitForCurrentGroup = totalRevenueForCurrentGroup - totalCostForCurrentGroup
    
    group_df = group_df.set_value(i, 'Total Bought', totalAmountForBuys)
    group_df = group_df.set_value(i, 'Total Sold', totalAmountForSells)
    group_df = group_df.set_value(i, 'Profit', totalProfitForCurrentGroup)
    group_df = group_df.set_value(i, 'Fee', totalFeeForCurrentGroup)
    group_df = group_df.set_value(i, 'Name', name)

    if totalProfitForCurrentGroup > 0 :
        resultDf = resultDf.append(group_df)
    else: 
        brokenDf = brokenDf.append(group_df)
        if totalProfitForCurrentGroup == 0.0:
            print('Group ' + name + ' had no round trip trades in this time period')
            print('\n')
        else:
            print('This trade was manually sold for a loss or ' + name + 
                  '. Total profit not added to final result is: ' + str(totalProfitForCurrentGroup))
            print('\n')

    i = i + 1
    
#loop over

Group ARKBTC had no round trip trades in this time period


Group BTGBTC had no round trip trades in this time period


Group DASHBTC had no round trip trades in this time period


Group ETCBTC had no round trip trades in this time period


Fixing relative cost for group HSRBTC


Fixing relative revenue for group ICNBTC


Fixing relative cost for group ICXBTC


Problems calculating group ICXBTC. Total profit not added to final result is: -0.0012021079999999518


Fixing relative revenue for group IOTABTC


Problems calculating group IOTABTC. Total profit not added to final result is: -0.00020700000000000232


Group LSKBTC had no round trip trades in this time period


Group LTCBTC had no round trip trades in this time period


Fixing relative revenue for group MCOBTC


Problems calculating group MCOBTC. Total profit not added to final result is: -0.001198170000000165


Fixing relative revenue for group NANOBTC


Fixing relative revenue for group NEBLBTC


Group NEOBTC had no round trip 

In [6]:
print('Successful calculation of below groups: If -, then that means we are counting all')
resultDf.fillna('-')

Successful calculation of below groups: If -, then that means we are counting all


,Fee,Name,Only counting,Profit,Total Bought,Total Sold
0,0.037370,ADABTC,-,0.001070,1563.000,1563.000
1,0.026768,AEBTC,-,0.000761,94.580,94.580
3,0.018729,BCCBTC,-,0.000443,0.201,0.201
5,0.056203,BTSBTC,-,0.000887,2991.000,2991.000
6,0.037144,CNDBTC,-,0.000612,5306.000,5306.000
8,0.037241,ELFBTC,-,0.000750,397.000,397.000
9,0.094337,EOSBTC,-,0.001560,70.560,70.560
11,0.012984,GASBTC,-,0.000328,6.940,6.940
12,0.018698,GRSBTC,-,0.000428,193.000,193.000
13,0.074696,HSRBTC,85.73,0.001568,85.730,85.730


In [7]:
profitDf = pd.DataFrame(columns=['Total Profit BTC', 'Total Fees BNB'])
profitDf['Total Profit BTC'] = [resultDf['Profit'].sum()]
profitDf['Total Fees BNB'] = [resultDf['Fee'].sum()]
profitDf.index = ['Total']

In [8]:
print('These were not added to final result. If all 0, then no round trip trades for these groups exist in the given time period. If running many pairs, this is expected.')
print('If negative, user manually sold for a loss and it is not counted, or the pair is currently in RT mode.')
brokenDf

These were not added to final result. If all 0, then no round trip trades for these groups exist in the given time period. If running many pairs, this is expected.
If negative, user manually sold for a loss and it is not counted, or the pair is currently in RT mode.


,Fee,Name,Only counting,Profit,Total Bought,Total Sold
2,0.000000,ARKBTC,NaN,0.000000,0.000,0.000
4,0.000000,BTGBTC,NaN,0.000000,0.000,0.000
7,0.000000,DASHBTC,NaN,0.000000,0.000,0.000
10,0.000000,ETCBTC,NaN,0.000000,0.000,0.000
15,0.109822,ICXBTC,356.200,-0.001202,356.200,356.200
17,0.013702,IOTABTC,138.000,-0.000207,138.000,276.000
20,0.000000,LSKBTC,NaN,0.000000,0.000,0.000
21,0.000000,LTCBTC,NaN,0.000000,0.000,0.000
22,0.054689,MCOBTC,68.160,-0.001198,68.160,68.160
26,0.000000,NEOBTC,NaN,0.000000,0.000,0.000


In [9]:
#USD Profits
usProfitDf = pd.DataFrame(index=[8000, 10000, 12000, 15000, 20000, 25000, 50000, 100000], 
                          columns=['USD Profit if cost of BTC is ...'])
usProfitDf['USD Profit if cost of BTC is ...'] = usProfitDf.index * (resultDf['Profit'].sum() - resultDf['Fees'])
usProfitDf.index.name = 'Price BTC'

KeyError: 'Fees'

In [ ]:
# Get profit in USD from real time BTC price from coinmarketcap

import requests

response = requests.get('https://api.coinmarketcap.com/v1/ticker/bitcoin/') 
response2 = requests.get('https://api.coinmarketcap.com/v1/ticker/binance-coin/') 
realTimeProfitDf = pd.DataFrame(index=['Total'])
if use_fees:
    realTimeProfitDf['USD Profit if cost of BTC is ...'] = float(response.json()[0]['price_usd']) * resultDf['Profit'].sum() - float(response2.json()[0]['price_usd']) * resultDf['Fee'].sum()
else:
    realTimeProfitDf['USD Profit if cost of BTC is ...'] = float(response.json()[0]['price_usd']) * resultDf['Profit'].sum()

In [ ]:
profitDf

In [ ]:
pd.options.display.float_format = '${:,.2f}'.format
print(usProfitDf.to_string(index=True))

In [ ]:
resultDf['USD Profit'] = (float(response.json()[0]['price_usd']) * resultDf['Profit'])
resultDf = resultDf.sort_values(by='USD Profit', ascending=False)
ax = resultDf.plot(x='Name', y='USD Profit', kind='bar', figsize=(16,7), title='Proft in USD per pair')
plt.show()
finalDf = pd.DataFrame(resultDf[['Name','USD Profit']], index=None)

print(finalDf.to_string(index=False))

In [ ]:
import datetime as datetime
print('\n')
print ('Current Bitcoin Price as of ' + datetime.datetime.now().strftime('%I:%M %p on %B %d, %Y') + ' is: ' + "${:,.2f}".format(float(response.json()[0]['price_usd'])))
print('\n')

In [ ]:
feeCost = float(response2.json()[0]['price_usd']) * float(resultDf['Fee'].sum())
label = str('Profit from ' + str(start) + ' to ' + str(end) + ' in USD')
realTimeProfitDf.columns=[label]
print('\n')
print(str('Fee\'s from BNB ' + str(start) + ' to ' + str(end) + ' currently costs ' + "${:,.2f}".format(feeCost) + ' in USD.'))
print('\n')
realTimeProfitDf

In [ ]:
end = pd.to_datetime(df['Date'].iloc[0])
start = pd.to_datetime(df['Date'].iloc[-1])

diff = end - start

days, seconds = diff.days, diff.seconds
if days == 0:
    days = 1
hours = days * 24.0 + seconds // 3600.0
minutes = hours * 60.0

priceBTC = float(response.json()[0]['price_usd'])
avgProfMin = resultDf['Profit'].sum() * priceBTC / minutes
avgProfHour = resultDf['Profit'].sum() * priceBTC / hours
avgProfDay = resultDf['Profit'].sum() * (priceBTC / days)

btcProfit = resultDf['Profit'].sum()
print('Average profit per minute is ' "${:,.2f}".format(avgProfMin) + ' or ' + "{:,.15f}".format(btcProfit/minutes) + ' BTC.')
print('Average profit per hour is ' "${:,.2f}".format(avgProfHour) + ' or ' + "{:,.15f}".format(btcProfit/hours) + ' BTC.')
print('Average daily profit is ' "${:,.2f}".format(avgProfDay) + ' or ' + "{:,.15f}".format(btcProfit/days) + ' BTC.')

print('Not bad for a bot trading for you!')